# GOES-R Level 1b Multiband Ingestion

## About

This notebook shows an example of ingesting multiple NetCDF files into a single TileDB group using TileDB-CF-Py. It can be a jumping point for constructing a GOES-R dataset tuned to your specific use case, or just a useful example for understanding the `NetCDF4ConverterEngine`.

## Requirements

### Data

This notebook uses GOES-R L1b radiance data that has been downloaded locally to a specific folder in the project repository. See the script [data/fetch_data.sh](data/fetch_data.sh) for the path hard-coded in this example.

### Software

This notebook was tested with Python 3.9.0, but should work with any Python version that is supported by all required libraries. It requires the following Python libraries:

* `tiledb`
* `tiledb-cf`
* `numpy`
* `netCDF4`

In [ ]:
import glob

import tiledb
import tiledb.cf
import numpy as np

In this example, we use the TileDB-CF NetCDF4ConverterEngine to combine a dataset of files.

Get all ABI files from a single hour. Split the files into separate lists depending on which group they were in.

In [ ]:
abi_files = tuple(glob.glob(f"data/noaa-goes17/ABI-L1b-RadF/2021/264/18/OR_ABI-L1b-RadF-M6C{band:02.0f}*.nc") for band in range(1, 12))
dim_dtype = np.uint32

Create a converter for the ABI files. Generate from a file, then view and modify.

In [ ]:
abi_converter = tiledb.cf.NetCDF4ConverterEngine.from_file(abi_files[0][0], dim_dtype=np.uint32)
abi_converter

## Add dimensions for file band and timestamp

In [ ]:
abi_converter.add_shared_dim("band", domain=(1, 12), dtype=dim_dtype)
abi_converter.add_shared_dim("time.index", domain=(0, 143), dtype=dim_dtype)
for array_creator in abi_converter:
    array_creator.domain.inject_dim_creator("band", 0)
    array_creator.domain.inject_dim_creator("time.index", 1)


In [ ]:
abi_converter.add_shared_dim(
    "midscan_time",
    domain=(np.datetime64("2000", "ms"), np.datetime64("2051", "ms")),
    dtype=np.dtype("datetime64[ms]"),
)
abi_converter.add_array_converter("time_axis_label", ("midscan_tile",), sparse=True, tiles=(1024,))